# 简介

## 定义

   **概率图模型**是一类用图形模式表达基于概率相关关系的模型的总称。概率图模型结合概率论与图论的知识，利用图来表示与模型有关的变量的联合概率分布。基本的图模型有**贝叶斯网络**(*Bayesian Network*)和**马尔可夫随机场**(*Markov Random Field*)。对应**有向图**和**无向图**。

## 目的

1.表达

* 使用无向图和有向图进行表达。 
* 使用动态和静态模型进行表达。

2.推断 

* 精确推断和模糊推断。 
* 在不确定性状态下进行决策。

3.学习 

* 自动学习参数和结构 
* 有完全数据情形和非完全数据情形。 

# 1.基本概念

## 1.概率分布(*Distribution*)

### 联合分布案例：
学生的智力$I$取值为$i^0$*(Low)*，$i^1$*(High)*；

试题难度$D$取值为$d^0$*(Easy)*，$d^1$*(Hard)*；

考试的得分为$G$，其取值为$g^1$*(A)*，$g^2$*(B)*，$g^3$*(C)*。

我们将变量之间组合，得到一个$2\times 2\times 3$种结果的因子表，这是一个联合分布$P(I,D,G)$。

|$I$   |$D$   |$G$   |$Probability$   |
|:--------:|:--------:|:--------:|:--------:|
|$i^0$|$d^0$|$g^1$|$0.126$
|$i^0$|$d^0$|$g^2$|$0.168$
|$i^0$|$d^0$|$g^3$|$0.126$
|$i^0$|$d^1$|$g^1$|$0.009$
|$i^0$|$d^1$|$g^2$|$0.045$
|$i^0$|$d^1$|$g^3$|$0.126$
|$i^1$|$d^0$|$g^1$|$0.252$
|$i^1$|$d^0$|$g^2$|$0.0224$
|$i^1$|$d^0$|$g^3$|$0.0056$
|$i^1$|$d^1$|$g^1$|$0.06$
|$i^1$|$d^1$|$g^2$|$0.036$
|$i^1$|$d^1$|$g^3$|$0.024$

### 条件分布(**Reduction**)
假设固定$G$的值为$g^1$，筛选出联合分布$P(I,D,g^1)$（注意此数据需要标准化才是一个概率）。

|$I$   |$D$   |$G$   |$Probability$   |
|:--------:|:--------:|:--------:|:--------:|
|$i^0$|$d^0$|$g^1$|$0.126$
|$i^0$|$d^1$|$g^1$|$0.009$
|$i^1$|$d^0$|$g^1$|$0.252$
|$i^1$|$d^1$|$g^1$|$0.06$

经过标准化（$\dfrac{Prob._{i}}{\sum_jProb._j}$），我们得到在$G$条件下$(I,D)$的分布$P(I,D\mid g^1)$

|$I$   |$D$   |$G$   |$Probability$   |
|:--------:|:--------:|:--------:|:--------:|
|$i^0$|$d^0$|$g^1$|$0.282$
|$i^0$|$d^1$|$g^1$|$0.02$
|$i^1$|$d^0$|$g^1$|$0.564$
|$i^1$|$d^1$|$g^1$|$0.134$

### 边缘化(**Normalization**)

将某一个变量求和，剩余变量的分布，例如边缘化$I$，$P(I,D)\rightarrow P(D)$

|$I$   |$D$    |$Probability$   |
|:--------:|:--------:|:--------:|
|$i^0$|$d^0$|$0.282$
|$i^0$|$d^1$|$0.02$
|$i^1$|$d^0$|$0.564$
|$i^1$|$d^1$|$0.134$

|$D$|$Probability$|
|:-----:|:------:|
|$d^0$|$0.846$|
|$d^1$|$0.154$|

## 2.因子(Factor)
* 因子是随机变量的函数$\phi(X_1,X_2,\dots,X_k)$；$P(I,D,G)$，$P(I,D\mid g^1)$都是因子
* 因子是处理概率分布的的基本手段。
* 因子是高维空间中用以定义概率分布的基本单元。
* 因子的*scope*是因子包含的变量：$scope=\{X_1,X_2,\dots,X_k\}$

### 因子的乘积
$$\phi_1(x,y)\times \phi_2(y,z)=\phi_3(x,y,z) $$
拥有相同的scope元素时，概率才能相乘。

### 因子的边缘化
$$\phi(x,z)=\int_y\phi(x,y,z)dy$$

### 因子的降维
$$\phi(x,y)=\phi(x,y,z)\Big|_{z=C}$$

# 2.贝叶斯网络基础

## 1.语义和因子分解
在上述案例下再加入两个变量$S$(*SAT score*)和$L$(*Recommendation letter*)。

$\require{AMScd}$
\begin{CD}
   I(P(I)) @>>> G(P(G\mid I,D)) @<<< D(P(D))\\
    @VVV @VVV \\
    S(P(S\mid I))  @.  L(P(L\mid G))
\end{CD}


上图就是一个简单的概率图模型，实质上是一系列的条件分布，变量的联合分布如下：


$$P(D,I,G,S,L)=P(D)P(I)P(G\mid I,D)P(S\mid I)P(L\mid G)$$

$P$代表一个因子，是贝叶斯网络$\mathscr{G}$的一种分解（可通过链式法则推导）。

贝叶斯网络：
* 节点代表随机变量的有向无环图*(Directed Acyclyc Graph)*。

* 对于任意节点$X_i$的条件概率，$P(X_i\mid Par_{\mathscr{G}}(X_i))$。$Par(\cdot)$代表父亲节点。

* **BN**表示联合分布$P(X_1,X_2,\dots,X_k)=\prod_iP(X_i\mid Par_{\mathscr{G}}(X_i))$

## 2.因果现象

在计算条件概率分布时，通常引入因果律，主要体现在**由因到果**和**由果到因**。

### 1.由因到果(Casual reasoning)

例如已知智力$I$和难度$D$会对得分$G$造成影响，得分$G$又会对推荐信$L$造成影响，据此得到：


$$P(l^1)\approx 0.5$$


$$P(l^1\mid i^0)\approx 0.39$$


$$P(l^1\mid i^0,d^0)\approx 0.51$$

### 2.由果到因(Evidential reasoning)

得到的结果可以更新先验知识，例如：
$$P(d^1)=0.4   \qquad    P(i^1)=0.3$$
在已知得分为$g^3$时条件分布发生变化：
$$P(d^1\mid g^3)\approx 0.63 \qquad P(i^1\mid g^3)\approx 0.08$$

### 3.混合推理(Intercasual reasoning)

原因之间可能会有交互影响，先顺着箭头推理，在逆着箭头推理，例如：

$$P(i^1)=0.3$$


$$P(i^1\mid g^2)\approx 0.175$$


$$P(i^1\mid g^2,d^1)\approx 0.34$$

也就是说，在获得$g^2$得分的条件下，高智商$i^1$的概率大幅度降低了，但是如果已知难度较大，高智商的概率反而比原来要更大。

## 3.概率影响的传播

### 1.变量$X$何时影响$Y$?


  $$ X \rightarrow Y  \\ 
   X\leftarrow Y \\
   X \rightarrow W \rightarrow Y  \\
   X\leftarrow W \leftarrow Y\\
   X \leftarrow W \rightarrow Y  \\ 
   X\rightarrow W \leftarrow Y $$


>**Active trail**：对贝叶斯网络中的一条路径$X_1-\dots-X_k$和观测变量集合$Z$，当$X_1$和$X_k$的取值能够相互影响时，称路径$X_1-\dots-X_k$是有效的。

以上情况都有可能出现$X$与$Y$相互影响的局面。要注意：

* 直接相连的变量一定不能条件独立

* 如果一条路径没有$V$型结构$X_{i-1}\rightarrow X_i \leftarrow X_{i+1}$，那么称路径$X_1- \dots - X_k$为**Active trail**（表示起点和终点不条件独立）。

### 2.在给定条件$Z$时变量$X$与$Y$的关系

在给定证据集$Z$时，变量的关系如下：

|Variable-Relations      |  $W\notin Z$   |$W\in Z$   |
|:------------------------------:|:------:|:------:|
|$ X \rightarrow W \rightarrow Y$|$\surd$ |$\times$|
|$X\leftarrow W \leftarrow Y$|$\surd$|$\times$|
|$X \leftarrow W \rightarrow Y$|$\surd$|$\times$|
|$X\rightarrow W \leftarrow Y $|$\times$|$\surd$|
  
   

单向流动：$$p(x,w,y)=p(x)p(w\mid x)p(y\mid w)$$

 $$\begin{align}
           p(y\mid x,w)&=\frac{p(x,w,y)}{p(x,w)}\\
                       &=\frac{p(x)p(w\mid x)p(y\mid w)}{p(w\mid x)p(x)}\\
                       &=p(y\mid w)\\
           \end{align}
                       $$
 
源流(**Conditional Independence**)：$$p(x,w,y)=p(w)p(x\mid w)p(y\mid w)$$ 

$$\begin{align}
           p(y\mid x,w)&=\frac{p(x,w,y)}{p(x,w)}\\
                       &=\frac{p(w)p(x\mid w)p(y\mid w)}{p(w)p(x\mid w)}\\
                       &=p(y\mid w)\\
           \end{align}
                       $$
                       
阻塞(**Marginal Independence**)：$$p(x,w,y)=p(x)p(y)p(w\mid x,y)$$

$$\begin{align}
   p(x,y)&=\frac{p(x,w,y)}{p(w\mid x,y)}\\
         &=\frac{p(x)p(y)p(w\mid x,y)}{p(w\mid x,y)}\\
         &=p(x)p(y)
   \end{align}$$

其中$\surd$代表$X$与$Y$之间相关，$\times$表示$X$与$Y$独立。**注意:如果$W$的子节点已知，相当于给出了$W$。**

在$Z$条件下$X_1-\dots-X_k$要成为*Active trail*，则满足：
* **(Activate V-structure)**对任意$V$型结构$X_{i-1}\rightarrow X_i \leftarrow X_{i+1}$，$X_i$或其后代节点$\in Z$。


* **(Valve opening)**所有不是$X_i$的节点$X_j$都满足$X_j\notin Z$（即路径上其他任何节点都未知）。

# 3.贝叶斯网络

## 1.条件独立

For events $\alpha,\beta,P\models \alpha\perp \beta$ if:

$$P(\alpha,\beta)=P(\alpha)P(\beta)$$

$$P(\alpha\mid\beta)=P(\alpha)$$

$$P(\beta\mid\alpha)=P(\beta)$$

For random variables $X,Y,Z,P\models (X\perp Y\mid Z)$ if:

$$P(X,Y\mid Z)=P(X\mid Z)P(Y\mid Z)$$

$$P(X\mid Y,Z)=P(X\mid Z)$$

$$P(Y\mid X,Z)=P(Y\mid Z)$$

$$P(X,Y,Z)\propto \phi_1(X,Z)\phi_2(Y,Z)$$

## 2.贝叶斯网络中的独立性

### 1.d-separation

>在给定$Z$的条件下，图$\mathscr{G}$中$X$与$Y$之间没有*active trail*，则称$X,Y$被d-separated，记为$d-sep_{\mathscr{G}}(X,Y\mid Z)$。

关于d-separation，有如下定理：

1.If $P$ factorizes over $\mathscr{G}$, and $d-sep_{\mathscr{G}}(X,Y\mid Z)$, then $P$ satisfies $(X\perp Y\mid Z)$.

（变量满足d-sep则条件独立）。

2.Any node is d-separated from its non-descendants given its parents.

3.If $P$ factorizes over $\mathscr{G}$,then in $P$,any variable is independent its non-descendants given its parents.

（变量节点会无条件依赖与父节点，但在父节点条件下条件独立于所有非后裔）。


### 2.I-map

>全局马尔可夫独立性集合：对于图$\mathscr{G}$，定义其与d-sep对应的独立性集合为$\mathscr{I}(\mathscr{G})=\{(X\perp Y\mid Z):d-sep_{\mathscr{G}}(X,Y\mid Z)\}$。

>I等价：两个图I等价即为两者独立性集合完全相同。

>I-map:如果$\mathscr{G}$满足$\mathscr{I}(P)$的独立性要求，即$\mathscr{I}(\mathscr{G})\subset\mathscr{I}(P)$，则图$\mathscr{G}$是$P$的一个I-map。通俗来说，图$\mathscr{G}$就是一个不连通的图。

独立性与分解的关系：

* 如果独立的概率分布$P$可以按照某个图$\mathscr{G}$分解，那么$\mathscr{G}$就是$P$的Imap。 

* 如果$\mathscr{G}$是概率分布$P$的Imap，那么$P$可以按照$\mathscr{G}$来进行分解。 


## 3.朴素贝叶斯(Naïve Bayes)

### 1.Bernoulli Naïve Bayes

一个类别$C$下有$X_1,X_2,\dots,X_n$个特征，一个朴素贝叶斯分类器有如下假设：

$$(X_i\perp X_j\mid C)\quad  for\quad all\quad X_i\quad and\quad X_j$$

根据链式法则：

$$P(C,X_i,\cdots,X_n)=P(C)\prod_{i=1}^nP(X_i\mid C)$$

对于二分类问题（只有在文档内部是否出现两种取值），给定观测下的类别比例：

$$\dfrac{P(C=c_1\mid x_1,x_2,\cdots,x_n)}{P(C=c_2\mid x_1,x_2,\cdots,x_n)}=\dfrac{P(C=c_1)}{P(C=c_2)}\prod_{i=1}^n\dfrac{P(x_i\mid C=c_1)}{P(x_i\mid C=c_2)}$$

也就是说类别概率等于先验概率之比乘以发生比(**Odd**)，发生比是一个似然函数。

### 2.Multinomial Naïve Bayes

每个变量的取值不再为是或否，而是在文档内部出现的次数，所有概率相加结果应该为1。结果相同：

$$\dfrac{P(C=c_1\mid x_1,x_2,\cdots,x_n)}{P(C=c_2\mid x_1,x_2,\cdots,x_n)}=\dfrac{P(C=c_1)}{P(C=c_2)}\prod_{i=1}^n\dfrac{P(x_i\mid C=c_1)}{P(x_i\mid C=c_2)}$$